In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/MyDrive/Colab Notebooks/PCA-CLS

/content/gdrive/MyDrive/Colab Notebooks/PCA-CLS


In [3]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=07195384cdf738ba606986b69aa971da65c35ab7efe563c83d9fea39d01eadfe
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:

decode_file = {
    'status':'decode',
    'raw_dir':'',
    'dset_dir':''
}
decode_model, decoder_raw = [], []

dataset_name = ['BC2GM', 'BC4CHEMD', 'BC5CDR-chem', 'BC5CDR-disease', 'JNLPBA', 'Linneaus', 'NCBI-disease', 's800']


In [5]:

def read_conll_2003(file_path):
    # sentences = []
    actual_tags, predict_tags = [], []
    with open(file_path, 'r', encoding='utf-8') as file:
        # current_sentence = []
        actual_tag, predict_tag = [], []
        for line in file:
            line = line.strip()
            if line:
                # Split the line into columns
                columns = line.split()
                actual_ner = columns[1]
                ner_label = columns[2]

                # Append the token information to the current sentence
                actual_tag.append(actual_ner)
                predict_tag.append(ner_label)
            else:
                # Blank line indicates the end of the current sentence
                if actual_tag:
                    actual_tags.append(actual_tag)
                    predict_tags.append(predict_tag)

                    actual_tag, predict_tag = [], []

    return actual_tags, predict_tags


In [6]:
def ScoreCalculator(raw_file = 'prediction/BC2GM/raw.out', score_file='Your Classification Report Name.csv'):
    y_true, y_pred = read_conll_2003(raw_file)
    print(classification_report(y_true, y_pred, digits=4, output_dict=True))

    clsf_report = pd.DataFrame(
        classification_report(y_true=y_true, y_pred=y_pred, output_dict=True)).transpose()
    clsf_report.to_csv(score_file, index=True)


In [7]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
import pandas as pd
import os

In [8]:

def PredictionDir_builder(dir):
    predict_path = os.path.join('prediction', dir)
    try:
        os.mkdir(predict_path)
    except FileExistsError:
        pass

    decode_file['raw_dir'] = os.path.join('dataset', dir, 'test.tsv')
    decode_file['dset_dir'] = os.path.join('model', dir, 'cnn_lstm.dset')
    for file in os.listdir(os.path.join('model', dir)):
        if file.endswith('model'):
            decode_model.append(os.path.join('model', dir, file))
            decoder_raw.append(os.path.join(predict_path, 'raw'+file+'.out'))



In [9]:
config_files = []

def Config_Gen():
    for dir in dataset_name:
        PredictionDir_builder(dir)
        for idx,model in enumerate(decode_model):
            temp_file = 'decode.'+"_".join(model.split('/')[1:]).split('.model')[0]+'.config'
            # print(temp_file)
            with open(os.path.join('decoder', temp_file), 'w', encoding='utf-8') as file:
                for key, value in decode_file.items():
                    file.write(f"{key}={value}\n")
                file.write("load_model_dir="+model+'\n')
                file.write("decode_dir="+decoder_raw[idx])

            config_files.append(temp_file)



In [10]:
Config_Gen()
print(config_files)


['decode.BC2GM_cnn_lstm.0.config', 'decode.BC2GM_cnn_lstm.0.config', 'decode.BC4CHEMD_cnn_lstm.0.config', 'decode.BC2GM_cnn_lstm.0.config', 'decode.BC4CHEMD_cnn_lstm.0.config', 'decode.BC5CDR-chem_cnn_lstm.0.config', 'decode.BC5CDR-chem_cnn_lstm.1.config', 'decode.BC5CDR-chem_cnn_lstm.4.config', 'decode.BC5CDR-chem_cnn_lstm.10.config', 'decode.BC5CDR-chem_cnn_lstm.26.config', 'decode.BC2GM_cnn_lstm.0.config', 'decode.BC4CHEMD_cnn_lstm.0.config', 'decode.BC5CDR-chem_cnn_lstm.0.config', 'decode.BC5CDR-chem_cnn_lstm.1.config', 'decode.BC5CDR-chem_cnn_lstm.4.config', 'decode.BC5CDR-chem_cnn_lstm.10.config', 'decode.BC5CDR-chem_cnn_lstm.26.config', 'decode.BC5CDR-disease_cnn_lstm.0.config', 'decode.BC2GM_cnn_lstm.0.config', 'decode.BC4CHEMD_cnn_lstm.0.config', 'decode.BC5CDR-chem_cnn_lstm.0.config', 'decode.BC5CDR-chem_cnn_lstm.1.config', 'decode.BC5CDR-chem_cnn_lstm.4.config', 'decode.BC5CDR-chem_cnn_lstm.10.config', 'decode.BC5CDR-chem_cnn_lstm.26.config', 'decode.BC5CDR-disease_cnn_lstm.

In [11]:
print(decoder_raw)

['prediction/BC2GM/rawcnn_lstm.0.model.out', 'prediction/BC4CHEMD/rawcnn_lstm.0.model.out', 'prediction/BC5CDR-chem/rawcnn_lstm.0.model.out', 'prediction/BC5CDR-chem/rawcnn_lstm.1.model.out', 'prediction/BC5CDR-chem/rawcnn_lstm.4.model.out', 'prediction/BC5CDR-chem/rawcnn_lstm.10.model.out', 'prediction/BC5CDR-chem/rawcnn_lstm.26.model.out', 'prediction/BC5CDR-disease/rawcnn_lstm.0.model.out', 'prediction/JNLPBA/rawcnn_lstm.0.model.out', 'prediction/JNLPBA/rawcnn_lstm.2.model.out', 'prediction/JNLPBA/rawcnn_lstm.6.model.out', 'prediction/JNLPBA/rawcnn_lstm.8.model.out', 'prediction/Linneaus/rawcnn_lstm.0.model.out', 'prediction/Linneaus/rawcnn_lstm.1.model.out', 'prediction/Linneaus/rawcnn_lstm.2.model.out', 'prediction/Linneaus/rawcnn_lstm.5.model.out', 'prediction/Linneaus/rawcnn_lstm.9.model.out', 'prediction/Linneaus/rawcnn_lstm.20.model.out', 'prediction/Linneaus/rawcnn_lstm.26.model.out', 'prediction/NCBI-disease/rawcnn_lstm.0.model.out', 'prediction/NCBI-disease/rawcnn_lstm.1.mo

In [12]:

for configFile in config_files:
    cmd = '!python Bio-NER/main.py --config decoder/'+configFile
    print(cmd)
    # os.system(cmd)


!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.4.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.10.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/m

In [15]:
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.4.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.10.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.4.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.10.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-disease_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.4.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.10.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-disease_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.6.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.8.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.4.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.10.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-disease_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.6.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.8.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.5.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.9.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.20.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.4.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.10.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-disease_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.6.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.8.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.5.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.9.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.20.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.NCBI-disease_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.NCBI-disease_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC2GM_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC4CHEMD_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.4.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.10.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-chem_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.BC5CDR-disease_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.6.config
!python Bio-NER/main.py --config decoder/decode.JNLPBA_cnn_lstm.8.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.5.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.9.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.20.config
!python Bio-NER/main.py --config decoder/decode.Linneaus_cnn_lstm.26.config
!python Bio-NER/main.py --config decoder/decode.NCBI-disease_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.NCBI-disease_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.0.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.1.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.2.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.3.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.6.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.9.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.15.config
!python Bio-NER/main.py --config decoder/decode.s800_cnn_lstm.28.config

Streaming output truncated to the last 5000 lines.
	size mismatch for word_hidden.wordrep.char_feature.char_embeddings.weight: copying a param with shape torch.Size([78, 30]) from checkpoint, the shape in current model is torch.Size([83, 30]).
	size mismatch for word_hidden.wordrep.word_embedding.weight: copying a param with shape torch.Size([16496, 300]) from checkpoint, the shape in current model is torch.Size([20190, 300]).
Seed num: 42
MODEL: decode
dataset/s800/test.tsv
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
DATA SUMMARY START:
 I/O:
     Start   Sequence   Laebling   task...
     Tag          scheme: BIO
     Split         token:  ||| 
     MAX SENTENCE LENGTH: 52
     MAX   WORD   LENGTH: -1
     Number   normalized: True
     Word  alphabet size: 20190
     Char  alphabet size: 83
     Label alphabet size: 4
     Word embedding  dir: None
     Char embedding  dir: None
     Word embedding size: 300
     Char embeddin

In [14]:

for predict_file in decoder_raw:
    ScoreCalculator(predict_file, os.path.split(predict_file)[1])



FileNotFoundError: [Errno 2] No such file or directory: 'prediction/BC2GM/rawcnn_lstm.0.model.out'